In [ ]:
import tensorflow as tf
from tensorflow import keras

In [4]:
model = keras.models.Sequential([
    keras.layers.Dense(units = 1, input_shape = [1]),
    keras.layers.Dense(units = 16, activation = 'relu'),
    keras.layers.Dense(units = 1)
])
model.compile(
    optimizer = 'sgd',
    loss = 'mean_squared_error',
)
model.fit(x = [-1, 0, 1], y = [-3, -1, 1], epochs = 5)

Epoch 1/5
1/1 [==============================] - 1s 605ms/step - loss: 3.9399
Epoch 2/5
1/1 [==============================] - 0s 42ms/step - loss: 3.8567
Epoch 3/5
1/1 [==============================] - 0s 19ms/step - loss: 3.7740
Epoch 4/5
1/1 [==============================] - 0s 18ms/step - loss: 3.6943
Epoch 5/5
1/1 [==============================] - 0s 14ms/step - loss: 3.6173


In [5]:
# generate a SavedModel
tf.saved_model.save(model, 'ss')

Convert a Keras Model

In [7]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model1 = converter.convert()

# Save the model
with open('model1.tflite', 'wb') as f:
    f.write(tflite_model1)

Convert a SavedModel (recommended)

In [8]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('/content/ss')
tflite_model2 = converter.convert()

# Save the model
with open('model2.tflite', 'wb') as f:
    f.write(tflite_model2)

Convert concrete functions

In [9]:
# Create a model using low-level tf.* APIs
class Squared(tf.Module):
    @tf.function(input_signature = [tf.TensorSpec(shape = [None], dtype = tf.float32)])
    def __call__(self, x):
        return tf.square(x)

model = Squared()
# (ro run your model) result = Squared(5.0) # This prints "25.0"
# (to generate a SavedModel) tf.saved_model.save(model, 'save_model_tf_dir')
concrete_func = model.__call__.get_concrete_function()

# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func], model)
tflite_model3 = converter.convert()

# Save the model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model3)